In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os
import pickle
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("bed_data.csv")

# Convert Date column to datetime and create Is_Weekend feature
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"])
    df["Is_Weekend"] = df["Date"].dt.dayofweek.isin([5, 6]).astype(int)  # 1 for Sat/Sun, 0 otherwise
    df.set_index("Date", inplace=True)

# Select features including Is_Weekend
features = [
    "Total_Admissions_Today",
    "Total_Discharges_Today",
    "Avg_LOS",
    "Avg_Age_Admissions_Today",
    "Total_Beds_Occupied_Today",
    "Is_Weekend"
]
target = "Total_Beds_Required_Tomorrow"

# Normalize numerical features (excluding Is_Weekend)
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[features[:-1] + [target]] = scaler.fit_transform(df[features[:-1] + [target]])  # Exclude Is_Weekend from scaling
df_scaled[features + [target]] = df_scaled[features + [target]].astype(np.float32)

# Train-Test Split (80% Train, 20% Test)
train_size = int(len(df_scaled) * 0.8)
train, test = df_scaled.iloc[:train_size], df_scaled.iloc[train_size:]

# Function to create sequences for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][features].values)  # Features
        y.append(data.iloc[i+seq_length][target])  # Target
    return np.array(X), np.array(y)

# Prepare sequences
seq_length = 10  # Model looks at last 10 days
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)

# Define LSTM Model
def build_lstm_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(seq_length, len(features))),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(loss='mse', optimizer='adam')
    return model

# Path for saving model as pickle
model_path = "lstm_bed_prediction.pkl"

# Load or Train Model
if os.path.exists(model_path):
    with open(model_path, "rb") as f:
        model_json, weights = pickle.load(f)
    model = model_from_json(model_json)
    model.set_weights(weights)
    model.compile(loss='mse', optimizer='adam')
    print("Loaded existing model.")
else:
    model = build_lstm_model()
    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)

    # Save model
    model_json = model.to_json()
    weights = model.get_weights()
    with open(model_path, "wb") as f:
        pickle.dump((model_json, weights), f)

    print("Trained and saved new model.")

# Predict next day's beds
latest_data = df_scaled.iloc[-seq_length:][features].values.reshape(1, seq_length, len(features))
predicted_scaled = model.predict(latest_data)

# Reverse scale the prediction
predicted_full = np.zeros((1, len(features)))
predicted_full[:, -1] = predicted_scaled.flatten()
predicted_beds = scaler.inverse_transform(predicted_full)[:, -1][0]  # Extract only target column

print(f"Predicted Beds for Tomorrow: {predicted_beds:.2f}")

def update_model_with_actual(actual_beds):
    """
    Updates the model with the new actual data at the end of the day.
    """
    global df_scaled, X_train, y_train

    # Add new actual data
    new_data = df.iloc[-1:].copy()
    new_data[target] = actual_beds  # Use actual entered value

    # Normalize new data (except Is_Weekend)
    new_data_scaled = new_data.copy()
    new_data_scaled[features[:-1] + [target]] = scaler.transform(new_data[features[:-1] + [target]])

    # Append new data
    df_scaled = pd.concat([df_scaled, new_data_scaled])  # Keep growing dataset

    # Re-create sequences
    X_train, y_train = create_sequences(df_scaled, seq_length)

    # Fine-tune model with the new data
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1)

    # Save updated model
    model_json = model.to_json()
    weights = model.get_weights()
    with open(model_path, "wb") as f:
        pickle.dump((model_json, weights), f)

    print("Model updated with new data.")

# Get actual value entered by hospital staff
actual_beds = float(input("Enter actual beds required for today: "))

# Compare and update
print(f"Predicted: {predicted_beds:.2f}, Actual: {actual_beds:.2f}")
update_model_with_actual(actual_beds)



c:\Users\HARSHDIP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0724 - val_loss: 0.0054
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0058 - val_loss: 0.0030
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0039 - val_loss: 0.0015
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0028 - val_loss: 4.5374e-04
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0034 - val_loss: 0.0015
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0027 - val_loss: 8.5694e-05
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0030 - val_loss: 5.3195e-04
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0029 - val_loss: 3.6316e-04
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0022 - val_loss: 1.9944e-04
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0028 - val_loss: 0.0021
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0032 - val_loss: 8.0308e-04
Epoch 12/100
36/36 ━━━━━━━━━